## Addresses

Create an `Address` object from a _bech32-encoded_ string:

In [2]:
from erdpy_core import Address

address = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

print("Address (bech32-encoded)", address.bech32())
print("Public key (hex-encoded):", address.hex())
print("Public key (hex-encoded):", address.pubkey.hex())

Address (bech32-encoded) erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1
Public key (hex-encoded): 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


... or from a _hex-encoded_ string - note that you have to provide the address prefix, also known as the **HRP** (_human-readable part_ of the address):

In [3]:
address = Address.from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1", "erd");

... or from a raw public key:

In [4]:
pubkey = bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = Address(pubkey, "erd")

Alternatively, you can use an `AddressFactory` (initialized with a specific **HRP**) to create addresses:

In [5]:
from erdpy_core import AddressFactory

factory = AddressFactory("erd")

address = factory.create_from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
address = factory.create_from_hex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1")
address = factory.create_from_pubkey(bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1"))

As seen above, addresses can be converted from one representation to another as follows:

In [6]:
print(address.bech32())
print(address.hex())

erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th
0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


... or using a converter:

In [7]:
from erdpy_core import AddressConverter

converter = AddressConverter("erd")

pubkey = converter.bech32_to_pubkey("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bech32 = converter.pubkey_to_bech32(bytes.fromhex("0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1"))

Getting the shard of an address:

In [8]:
print("Shard:", address.get_shard())

Shard: 1


Checking whether an address is a smart contract:

In [9]:
address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

print("Is contract:", address.is_smart_contract())

Is contract: True


## EGLD / ESDT transfers

Create an EGLD transfer:

In [10]:
from erdpy_core import Address, TokenPayment, Transaction

tx = Transaction(
    nonce=90,
    sender=Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"),
    receiver=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
    value=TokenPayment.egld_from_amount("1.0"),
    gas_limit=50000,
    gas_price=1000000000,
    chain_id="D",
    version=1
)

print(tx.to_dictionary())

OrderedDict([('nonce', 90), ('value', '1000000000000000000'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 50000), ('chainID', 'D'), ('version', 1), ('signature', '')])


We'll see later how to [sign](#signing-objects) and [broadcast](#broadcasting-transactions) a transaction.

Create an EGLD transfer, but this time with a payload (data):

In [11]:
from erdpy_core import TransactionPayload

data = TransactionPayload.from_str("for the book")

tx = Transaction(
    nonce=91,
    sender=Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"),
    receiver=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
    value=TokenPayment.egld_from_amount("3.0"),
    data=data,
    gas_limit=50000 + 1500 * data.length(),
    gas_price=1000000000,
    chain_id="D",
    version=1
)

print(tx.to_dictionary())

OrderedDict([('nonce', 91), ('value', '3000000000000000000'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 68000), ('data', 'Zm9yIHRoZSBib29r'), ('chainID', 'D'), ('version', 1), ('signature', '')])


Alternatively, we can create an EGLD transfer using a **transaction builder** (as we will see below, transaction builders are more commonly used). But before that, we have to create a configuration object (for any builder that we might use):

In [12]:
from erdpy_core.transaction_builders import DefaultTransactionBuildersConfiguration

config = DefaultTransactionBuildersConfiguration(chain_id="D")

The **transaction builder** is parametrized at instantiation, and the transaction is obtained by invoking the `build()` method:

In [13]:
from erdpy_core.transaction_builders import EGLDTransferBuilder

alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")
payment = TokenPayment.egld_from_amount("1.00")

# With "data" field
builder = EGLDTransferBuilder(
    config=config,
    sender=alice,
    receiver=bob,
    payment=payment,
    data="for the book",
    # Nonce can be set now, or later, prior signing
    nonce=42
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 42), ('value', '1000000000000000000'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 68000), ('data', 'Zm9yIHRoZSBib29r'), ('chainID', 'D'), ('version', 1), ('signature', '')])
Transaction data: for the book


Create a single ESDT transfer:

In [14]:
from erdpy_core.transaction_builders import ESDTTransferBuilder

payment = TokenPayment.fungible_from_amount("COUNTER-8b028f", "100.00", 2)

builder = ESDTTransferBuilder(
    config=config,
    sender=alice,
    receiver=bob,
    payment=payment
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 419000), ('data', 'RVNEVFRyYW5zZmVyQDQzNGY1NTRlNTQ0NTUyMmQzODYyMzAzMjM4NjZAMjcxMA=='), ('chainID', 'D'), ('version', 1), ('signature', '')])
Transaction data: ESDTTransfer@434f554e5445522d386230323866@2710


Create a single NFT transfer:

In [15]:
from erdpy_core.transaction_builders import ESDTNFTTransferBuilder

payment = TokenPayment.non_fungible("ERDPY-38f249", 1)

builder = ESDTNFTTransferBuilder(
    config=config,
    sender=alice,
    destination=bob,
    payment=payment
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 1216500), ('data', 'RVNEVE5GVFRyYW5zZmVyQDQ1NTI0NDUwNTkyZDMzMzg2NjMyMzQzOUAwMUAwMUA4MDQ5ZDYzOWU1YTY5ODBkMWNkMjM5MmFiY2NlNDEwMjljZGE3NGExNTYzNTIzYTIwMmYwOTY0MWNjMjYxOGY4'), ('chainID', 'D'), ('version', 1), ('signature', '')])
Transaction data: ESDTNFTTransfer@45524450592d333866323439@01@01@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a single SFT transfer (almost same as above):

In [16]:
payment = TokenPayment.semi_fungible("SEMI-9efd0f", 1, 5)

builder = ESDTNFTTransferBuilder(
    config=config,
    sender=alice,
    destination=bob,
    payment=payment
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 1213500), ('data', 'RVNEVE5GVFRyYW5zZmVyQDUzNDU0ZDQ5MmQzOTY1NjY2NDMwNjZAMDFAMDVAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOA=='), ('chainID', 'D'), ('version', 1), ('signature', '')])
Transaction data: ESDTNFTTransfer@53454d492d396566643066@01@05@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Create a multiple ESDT / NFT transfer:

In [17]:
from erdpy_core.transaction_builders import MultiESDTNFTTransferBuilder

payment_one = TokenPayment.non_fungible("ERDPY-38f249", 1)
payment_two = TokenPayment.fungible_from_amount("BAR-c80d29", "10.00", 18)

builder = MultiESDTNFTTransferBuilder(
    config=config,
    sender=alice,
    destination=bob,
    payments=[payment_one, payment_two]
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 2287000), ('data', 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAODA0OWQ2MzllNWE2OTgwZDFjZDIzOTJhYmNjZTQxMDI5Y2RhNzRhMTU2MzUyM2EyMDJmMDk2NDFjYzI2MThmOEAwMkA0NTUyNDQ1MDU5MmQzMzM4NjYzMjM0MzlAMDFAMDFANDI0MTUyMmQ2MzM4MzA2NDMyMzlAQDhhYzcyMzA0ODllODAwMDA='), ('chainID', 'D'), ('version', 1), ('signature', '')])
Transaction data: MultiESDTNFTTransfer@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8@02@45524450592d333866323439@01@01@4241522d633830643239@@8ac7230489e80000


## Contract deployments and interactions

Create a transaction to deploy a smart contract:

In [18]:
from pathlib import Path

from erdpy_core import CodeMetadata
from erdpy_core.transaction_builders import ContractDeploymentBuilder

metadata = CodeMetadata(upgradeable=True, readable=True, payable=True, payable_by_contract=True)

builder = ContractDeploymentBuilder(
    config,
    owner=alice,
    deploy_arguments=[42, "test"],
    code_metadata=metadata,
    code=Path("./contracts/counter.wasm").read_bytes(),
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqqq6gq4hu'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'MDA2MTczNmQwMTAwMDAwMDAxMTUwNDYwMDM3ZjdmN2UwMTdmNjAwMjdmN2YwMTdlNjAwMTdlMDA2MDAwMDAwMjQyMDMwMzY1NmU3NjExNjk2ZTc0MzYzNDczNzQ2ZjcyNjE2NzY1NTM3NDZmNzI2NTAwMDAwMzY1NmU3NjEwNjk2ZTc0MzYzNDczNzQ2ZjcyNjE2NzY1NGM2ZjYxNjQwMDAxMDM2NTZlNzYwYjY5NmU3NDM2MzQ2NjY5NmU2OTczNjgwMDAyMDMwNTA0MDMwMzAzMDMwNDA1MDE3MDAxMDEwMTA1MDMwMTAwMDIwNjA4MDE3ZjAxNDE5MDg4MDQwYjA3MmYwNTA2NmQ2NTZkNmY3Mjc5MDIwMDA0Njk2ZTY5NzQwMDAzMDk2OTZlNjM3MjY1NmQ2NTZlNzQwMDA0MDk2NDY1NjM3MjY1NmQ2NTZlNzQwMDA1MDM2NzY1NzQwMDA2MGE4YTAxMDQxMzAwNDE4MDg4ODA4MDAwNDEwNzQyMDExMDgwODA4MDgwMDAxYTBiMmUwMTAxN2U0MTgwODg4MDgwMDA0MTA3NDE4MDg4ODA4MDAwNDEwNzEwODE4MDgwODAwMDQyMDE3YzIyMDAxMDgwODA4MDgwMDAxYTIwMDAxMDgyODA4MDgwMDAwYjJlMDEwMTdlNDE4MDg4ODA4MDAwNDEwNzQxODA4ODgwODAwMDQxMDcxMDgxODA4MD

Create a transaction to upgrade an existing smart contract:

In [19]:
from erdpy_core.transaction_builders import ContractUpgradeBuilder

contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")
owner = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
metadata = CodeMetadata(upgradeable=True, readable=True, payable=True, payable_by_contract=True)

builder = ContractUpgradeBuilder(
    config,
    contract=contract_address,
    owner=owner,
    upgrade_arguments=[42, "test"],
    code_metadata=metadata,
    code=Path("./contracts/counter.wasm").read_bytes(),
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'dXBncmFkZUNvbnRyYWN0QDAwNjE3MzZkMDEwMDAwMDAwMTE1MDQ2MDAzN2Y3ZjdlMDE3ZjYwMDI3ZjdmMDE3ZTYwMDE3ZTAwNjAwMDAwMDI0MjAzMDM2NTZlNzYxMTY5NmU3NDM2MzQ3Mzc0NmY3MjYxNjc2NTUzNzQ2ZjcyNjUwMDAwMDM2NTZlNzYxMDY5NmU3NDM2MzQ3Mzc0NmY3MjYxNjc2NTRjNmY2MTY0MDAwMTAzNjU2ZTc2MGI2OTZlNzQzNjM0NjY2OTZlNjk3MzY4MDAwMjAzMDUwNDAzMDMwMzAzMDQwNTAxNzAwMTAxMDEwNTAzMDEwMDAyMDYwODAxN2YwMTQxOTA4ODA0MGIwNzJmMDUwNjZkNjU2ZDZmNzI3OTAyMDAwNDY5NmU2OTc0MDAwMzA5Njk2ZTYzNzI2NTZkNjU2ZTc0MDAwNDA5NjQ2NTYzNzI2NTZkNjU2ZTc0MDAwNTAzNjc2NTc0MDAwNjBhOGEwMTA0MTMwMDQxODA4ODgwODAwMDQxMDc0MjAxMTA4MDgwODA4MDAwMWEwYjJlMDEwMTdlNDE4MDg4ODA4MDAwNDEwNzQxODA4ODgwODAwMDQxMDcxMDgxODA4MDgwMDA0MjAxN2MyMjAwMTA4MDgwODA4MDAwMWEyMDAwMTA4MjgwODA4MDAwMGIyZTAxMDE3ZTQxODA4ODgwODAwMDQxMDc0MTgwODg4MD

Create a transaction that invokes a smart contract function:

In [20]:
from erdpy_core.transaction_builders import ContractCallBuilder

contract_address = Address.from_bech32("erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm")

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="foo",
    caller=alice,
    call_arguments=[42, "test", bob],
    gas_limit=10000000
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qqqqqqqqqqqqqpgquzmh78klkqwt0p4rjys0qtp3la07gz4d396qn50nnm'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'Zm9vQDJhQDc0NjU3Mzc0QDgwNDlkNjM5ZTVhNjk4MGQxY2QyMzkyYWJjY2U0MTAyOWNkYTc0YTE1NjM1MjNhMjAyZjA5NjQxY2MyNjE4Zjg='), ('chainID', 'D'), ('version', 1), ('signature', '')])
Transaction data: foo@2a@74657374@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


Now, let's create a call that also transfer one or more tokens (**transfer & execute**):

In [21]:
transfers = [
    TokenPayment.non_fungible("ERDPY-38f249", 1),
    TokenPayment.fungible_from_amount("BAR-c80d29", "10.00", 18)
]

builder = ContractCallBuilder(
    config,
    contract=contract_address,
    function_name="hello",
    caller=alice,
    call_arguments=[42, "test", bob],
    gas_limit=10000000,
    esdt_transfers=transfers
)

tx = builder.build()
print("Transaction:", tx.to_dictionary())
print("Transaction data:", tx.data)

Transaction: OrderedDict([('nonce', 0), ('value', '0'), ('receiver', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('sender', 'erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th'), ('gasPrice', 1000000000), ('gasLimit', 10000000), ('data', 'TXVsdGlFU0RUTkZUVHJhbnNmZXJAMDAwMDAwMDAwMDAwMDAwMDA1MDBlMGI3N2YxZWRmYjAxY2I3ODZhMzkxMjBmMDJjMzFmZjVmZTQwYWFkODk3NEAwMkA0NTUyNDQ1MDU5MmQzMzM4NjYzMjM0MzlAMDFAMDFANDI0MTUyMmQ2MzM4MzA2NDMyMzlAQDhhYzcyMzA0ODllODAwMDBANjg2NTZjNmM2ZkAyYUA3NDY1NzM3NEA4MDQ5ZDYzOWU1YTY5ODBkMWNkMjM5MmFiY2NlNDEwMjljZGE3NGExNTYzNTIzYTIwMmYwOTY0MWNjMjYxOGY4'), ('chainID', 'D'), ('version', 1), ('signature', '')])
Transaction data: MultiESDTNFTTransfer@00000000000000000500e0b77f1edfb01cb786a39120f02c31ff5fe40aad8974@02@45524450592d333866323439@01@01@4241522d633830643239@@8ac7230489e80000@68656c6c6f@2a@74657374@8049d639e5a6980d1cd2392abcce41029cda74a1563523a202f09641cc2618f8


## Contract queries

In order to create a contract query and run it against a network provider (more details about **network providers** can be found below), do as follows:

In [22]:
from erdpy_core import ContractQueryBuilder
from erdpy_core.interfaces import IAddress
from erdpy_network import ApiNetworkProvider

contract: IAddress = Address.from_bech32("erd1qqqqqqqqqqqqqpgqnzm7yhayarylux045qlm4lgzmtcsgrqg396qr9kupx")

builder = ContractQueryBuilder(
    contract=contract,
    function="getSum",
    call_arguments=[],
    caller=alice
)

query = builder.build()

network_provider = ApiNetworkProvider("https://devnet-api.elrond.com")
response = network_provider.query_contract(query)

print("Return code:", response.return_code)
print("Return data:", response.return_data)

Return code: ok
Return data: ['LQ==']


## Creating wallets

Mnemonic generation is based on [`trezor/python-mnemonic`](https://github.com/trezor/python-mnemonic) and can be achieved as follows:

In [23]:
from erdpy_wallet import Mnemonic

mnemonic = Mnemonic.generate()
words = mnemonic.get_words()

print(words)

['gravity', 'chat', 'aerobic', 'gift', 'few', 'popular', 'extend', 'nuclear', 'sure', 'math', 'avocado', 'convince', 'insect', 'outdoor', 'obscure', 'pledge', 'side', 'crawl', 'surge', 'glove', 'practice', 'celery', 'depend', 'plug']


Given a mnemonic, one can derive keypairs:

In [24]:
secret_key = mnemonic.derive_key(0)
public_key = secret_key.generate_public_key()

print("Secret key", secret_key.hex())
print("Public key", public_key.hex())

Secret key de6f77ec1007e3027891983e2188201e5a535824ce627d9fcbde944b7ee3b358
Public key 82cfe5e49c08108ec637d0999a0161acb13af1f3faf24553d84182fc6e516957


A keypair can be saved as a JSON wallet (recommended):

In [25]:
from erdpy_wallet import UserWallet

wallet = UserWallet(secret_key, "password")
wallet.save(Path("./output/wallet.json"), address_hrp="erd")

... or as a PEM wallet (usually not recommended):

In [26]:
from erdpy_wallet import UserPEM

label = Address(public_key.buffer, "erd").bech32()
pem = UserPEM(label=label, secret_key=secret_key)
pem.save(Path("./output/wallet.pem"))

## Loading wallets

 This is not a very common use-case - refer to [signing objects](#signing-objects) instead.

From a JSON wallet:

In [27]:
from erdpy_wallet import UserWallet

secret_key = UserWallet.decrypt_secret_key_from_file(Path("./testwallets/alice.json"), "password")
public_key = secret_key.generate_public_key()

print("Secret key", secret_key.hex())
print("Public key", public_key.hex())

Secret key 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Public key 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


From a PEM file:

In [28]:
from erdpy_wallet import UserPEM

pem = UserPEM.from_file(Path("./testwallets/alice.pem"))

print("Secret key", pem.secret_key.hex())
print("Public key", pem.public_key.hex())

Secret key 413f42575f7f26fad3317a778771212fdb80245850981e48b58a4f25e344e8f9
Public key 0139472eff6886771a982f3083da5d421f24c29181e63888228dc81ca60d69e1


## Signing objects

Creating a `UserSigner` from a JSON wallet:

In [29]:
from erdpy_wallet import UserSigner

signer = UserSigner.from_wallet(Path("./testwallets/alice.json"), "password")

Creating a `UserSigner` from a PEM file:

In [30]:
signer = UserSigner.from_pem_file(Path("./testwallets/alice.pem"))

Signable objects (messages, transactions) must adhere to the following interface:

```
class ISignable(Protocol):
    def serialize_for_signing(self) -> bytes: ...
```

Both `Transaction` and `Message` - defined in `erdpy_core` - implement `ISignable`.

Signing a transaction:

In [31]:
from erdpy_core import Transaction

tx = Transaction(
    nonce=90,
    sender=Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th"),
    receiver=Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx"),
    value=TokenPayment.egld_from_amount("1.0"),
    gas_limit=50000,
    gas_price=1000000000,
    chain_id="D",
    version=1
)

tx.signature = signer.sign(tx)
print("Signature", tx.signature.hex())

Signature 762e224eb5b8f2e801c2f0ea5ca1fb0aa08d4566a04f3846b250e222b76cd5d2141f5f131afecda9a3c5433f41790f33c45a559bea9a7d28ba964308fbcbfd0e


Signing an arbitrary message:

In [32]:
from erdpy_core import Message

message = Message.from_string("hello")
message.signature = signer.sign(message)

print("Signature", message.signature.hex())

Signature 66dd503199222d3104cb5381da953b14c895ca564579f98934c4e54a45d75f097da2a0c30060c0e7d014928b8e54509335cadc69b2bb1940cd59bd0c1bb80b0b


## Verifying signatures

Creating a `UserVerifier`:

In [33]:
from erdpy_core import Address
from erdpy_wallet import UserVerifier

alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")
bob = Address.from_bech32("erd1spyavw0956vq68xj8y4tenjpq2wd5a9p2c6j8gsz7ztyrnpxrruqzu66jx")
alice_verifier = UserVerifier.from_address(alice)
bob_verifier = UserVerifier.from_address(bob)

For objects to support signature verification, they must adhere to the following interface:

```
class IVerifiable(Protocol):
    signature: ISignature
    def serialize_for_signing(self) -> bytes: ...
```

Both `Transaction` and `Message` - defined in `erdpy_core` - implement `IVerifiable`.

Verifying a signature:

In [34]:
print(f"Is signature of Alice?", alice_verifier.verify(tx))
print(f"Is signature of Alice?", alice_verifier.verify(message))
print(f"Is signature of Bob?", bob_verifier.verify(tx))
print(f"Is signature of Bob?", bob_verifier.verify(message))

Is signature of Alice? True
Is signature of Alice? True
Is signature of Bob? False
Is signature of Bob? False


## Creating network providers

It's recommended to use the `erdpy_network` components **as a starting point**. As your application matures, switch to using your own network provider (e.g. deriving from the default ones), tailored to your requirements.

Creating an API provider:

In [35]:
from erdpy_network import ApiNetworkProvider

provider = ApiNetworkProvider("https://devnet-api.elrond.com");

Creating a Proxy provider:

In [36]:
from erdpy_network import ProxyNetworkProvider

provider = ProxyNetworkProvider("https://devnet-gateway.elrond.com");

## Fetching network parameters

In order to fetch network parameters, do as follows:

In [37]:
config = provider.get_network_config();

print("Chain ID", config.chain_id);
print("Min gas price:", config.min_gas_price);

Chain ID D
Min gas price: 1000000000


## Fetching account state

The following snippet fetches (from the Network) the **nonce** and the **balance** of an account:

In [38]:
account_on_network = provider.get_account(alice)

print("Nonce", account_on_network.nonce)
print("Balance", account_on_network.balance)

Nonce 11457
Balance 4795847845371071841


When sending a number of transactions, you usually have to first fetch the account nonce from the network (see above), then manage it locally (e.g. increment upon signing & broadcasting a transaction):

In [39]:
from erdpy_core import AccountNonceHolder

nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx.nonce = nonce_holder.get_nonce_then_increment()
# Then, sign transaction & broadcast the transaction.

For further reference, please see [nonce management](https://docs.elrond.com/integrators/creating-transactions/#nonce-management).

## Broadcasting transactions

Broadcast a single transaction (we are depicting the whole flow below):

In [40]:
alice = Address.from_bech32("erd1qyu5wthldzr8wx5c9ucg8kjagg0jfs53s8nr3zpz3hypefsdd8ssycr6th")

tx = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)

tx.nonce = alice_on_network.nonce
tx.signature = signer.sign(tx)

hash = provider.send_transaction(tx)
print("Transaction hash:", hash)

Transaction hash: 8d6f52388a35b16277d5cb0100ba87595286218f683970230acfc4c0bcf205d0


Broadcast multiple transactions:

In [41]:
tx_1 = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

tx_2 = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

tx_3 = Transaction(
    sender=alice,
    receiver=alice,
    gas_limit=50000,
    chain_id="D"
)

alice_on_network = provider.get_account(alice)
nonce_holder = AccountNonceHolder(account_on_network.nonce)

tx_1.nonce = nonce_holder.get_nonce_then_increment()
tx_2.nonce = nonce_holder.get_nonce_then_increment()
tx_3.nonce = nonce_holder.get_nonce_then_increment()

tx_1.signature = signer.sign(tx_1)
tx_2.signature = signer.sign(tx_2)
tx_3.signature = signer.sign(tx_3)

hashes = provider.send_transactions([tx_1, tx_2, tx_3])
print("Transactions hashes:", hashes)

Transactions hashes: (3, {'0': '8d6f52388a35b16277d5cb0100ba87595286218f683970230acfc4c0bcf205d0', '1': 'ad6b8aa4ccada227b073ed4a1f792f1349c47b6a6496777e2514fa0dac1bedcf', '2': '5ad07236b0139c1f2274d76cebc508b327a8ab5e963634e068369c52a3dd2cce'})


Now let's fetch a previously-broadcasted transaction:

In [42]:
tx_on_network = provider.get_transaction("09e3b68d39f3759913239b927c7feb9ac871c8877e76bc56e1be45a2a597eb53")
print("Status:", tx_on_network.status)
print("Is completed:", tx_on_network.is_completed)

Status: success
Is completed: True
